# Introduction to Time Series Anaysis

### Data Science 350
### Stephen Elston

## Introduciton

This notebook provides an overview of time series analysis. Time series are an extremely common data type. Just a few of the many applications of time series analysis include:

- **Demand forecasting:** Electricity production, Internet bandwidth, Traffic management, Inventory management
- **Medicine:** Time dependent treatment effects, EKG, EEG
- **Engineering and Science:** Signal analysis, Analysis of physical processes
- **Capital markets and economics:** Seasonal unemployment, Price/return series, Risk analysis

In this lesson you will learn the following:

- Basic properties of time series.
- How to perform and understand decomposition of time series.
- Modeling of time series residuals and the ARIMA model.
- Forecasting time series values from models. 

As you work with time series keep in mind the wise words of the famous american baseball player and team manager, Yogi Berra; 

<center> **“It's tough to make predictions, especially about the future.”**!</center>

****
**Reference:** Here is a selection of references you can use to go deeper into time series analysis with Python:

1. If you would like more information and examples for working with time series data in Python Pandas look [here](https://pandas.pydata.org/pandas-docs/stable/timeseries.html)
2. Time Series in Python: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/
3. ARIMA in Python: http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/
4. ARIMA models tutorial from the 2011 Scipy conference: https://conference.scipy.org/scipy2011/slides/mckinney_time_series.pdf
5. Cross-Validating Time Series Models: https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/
6. Exponential Smoothing Forecasting: https://grisha.org/blog/2016/01/29/triple-exponential-smoothing-forecasting/
7. Tune Hyper-Parameters in Keras/Scikit-Learn: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
****

## Short History of Time Series Analysis

The history of time series analysis starts with the pionering work of George Udny Yule (1927) and Gilber Walker (1931). Both Yule and Walker worked on the auto regresive (AR) model for stochastic time series.

![](img/George_Udny_Yule.jpg)
<center> **George Yule; time series pioner**

Mathematical prodegee, Norbert Weiner invented filters for stochastic time series processes during the Second World war. Weiner worked at MIT and was assigned to a project to improve the accuracy of anti-aircraft guns using the noisy radar signals of the day. He published his seminal paper on the subject in 1949. If you have recently used a mobile phone or streamed video or audio you are reciving benefits of wiener's research!

![](img/Norbert_wiener.jpg)
<center> **Norbert Weiner: Invented time series filter**

George Box and Gwilym Jenkins fully developed the the statistical theory  of time series by extending the work of Yule and Walker in the 1950s and 1960s. This work was fully developed in thier seminal 1970  book. Their theory included the auto regressive moving averge (ARMA) model and the auto regressive integrfated moving averge (ARIMA) models we use in this notebook.

George Box was maried to Joan Fisher Box, an outstanding statistician in her own right and daughter of Ronald Fisher. 

![](img/GeorgeEPBox.jpg)
<center> **George Box fully developed the ARIMA model**

![](img/BoxJenkins.jpg)
<center> **Seminal book: by Box and  Jenkins**

## Working with Time Series in Pandas

Pandas has significant capabilities for working with time series data. The key is the index of the Pandas series or data frame. The index contains the date-time information or time interval information. 

Let's start with a simple univariate time series. A univariate time series can be represented as a Pandas series, with the appropriate index. 

The code in the cell below creates a series of sinusoidal values. Execute this code and examine the results.

In [ ]:
from math import sin
import pandas as pd
ts = pd.Series([sin(x/30.0) for x in range(366)])
ts.head(20)

The object you created is a series of floating point values. However, this is not yet a time series. For that, we need to add an index.

The code in the cell below adds a new set of index values to the Pandas series. Execute this code and examine the results. 

In [ ]:
ts.index = pd.date_range(start = '1-1-2016', end = '12-31-2016', freg = 'D')
ts.head(20)

You can see that the index is now the date-time for each value. This Pandas series is now an actual time series. 

Let's plot the time series. The code in the cell below plots the values of the time series against the index. Notice that there is no need to explicitly specify the values for the x axis as these index is implied. Execute the code and examine the results. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
ts.plot()
plt.title('A simple time series plot')
plt.ylabel('Value')
plt.xlabel('Date')

You can see that the time axis is labeled automatically. 

Pandas provides many methods to manipulate and transform time series. For example, one can subset a time series using a range of time values from the index. The code in the cell below takes a subset of the time series by specifying a date range and displaying a plot. Execute this code and examine the result. 

In [ ]:
ts['1/1/2016':'6/30/2016'].plot()
plt.title('A simple time series plot')
plt.ylabel('Value')
plt.xlabel('Date')

Notice that the plot covers the subset of dates specified. 

## Basic Time Series Properties

In this section we will explore some basic properties time series. Understnding these properties will help in understanding the models we explore in subsequent sections.

### Properties of White Noise Series

A random series drawn from **independent identically distributed (iid)** noise drawen from a Normal distribution. Such a series is said to be a **white noise** series. Since the series is iid there is no correlation from one value to the next. We can write a **discrete** white noise time series as just:

$$X(t) = (w_1, w_2, w_3, \dots, w_n)\\
where\\
w_t = N(0, \theta)$$

Notice that the standard deviation and therefore the variance of the series, $\theta$, is constant in time. We say that a time series with a constant variance is **stationary**. The properties of a stationary time series do not vary with time. 

Further, the values of the time series are given at specific or discrete times, making this a discrete time series. In compuational time series analysis we nearly always work with discrete time series. Some time series are inherently discrete including, unemployment rate average over a month, the daily closing price of a stock. Even if the underlying time series is continious, we typically work with **values sampled at discrete points in time**. For example, temperature is a continious variable, but we will generally work with sampled variables, such as hourly measurements. 

The code in the cell below creates a time series from an iid Normal distribution with mean zero. Execute this code and note the attributes and the plot.

In [ ]:
def plot_ts(ts, lab = ''):
    ts.plot()
    plt.title('Time series plot of ' + lab)
    plt.ylabel('Value')
    plt.xlabel('Date')

import numpy.random as nr
nr.seed(3344)
white = pd.Series(nr.normal(size = 730),
                 index = pd.date_range(start = '1-1-2014', end = '12-31-2015', freg = 'D'))
plot_ts(white, 'white noise')

Notice that the values of the time series move wander randomly around zero, with no particular trend. 

Next, let's look at the distribution of the time series values. The code in the cell below plots the histogram and Q-Q Normal plot of the values of the time series. Run this code and examine the results.  

In [ ]:
def dist_ts(ts, lab = '', bins = 40):
    import scipy.stats as ss
    ## Setup a figure with two subplots side by side
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))
    ## Plot the histogram with labels
    ts.hist(ax = ax1, bins = bins, alpha = 0.5)
    ax1.set_xlabel('Value')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Histogram of ' + lab)
    ## Plot the q-q plot on the other axes
    ss.probplot(ts, plot = ax2)
    
dist_ts(white, 'white noise')    

As expected, the values of the white noise series are normalilly distributed. When examining these plots keep in mind there are only 365 values, so we should expect quite a lot of random variation. 

The values of the white noise series are iid, so we do not expect the values to show any dependency over time. In time series analysis we measure dependency using **autocorrelation**. Autocorrelation is the correlation of a series with itself lagged (offset in time) by some number of time steps. Autocorrelation at lag k can be writen as:

$$\rho_k = \frac{\gamma_k}{n \sigma^2} = \frac{1}{n \sigma^2} {\Sigma_{t = 1}^N (x_{t} - \mu) \cdot (x_{t-k} - \mu)}\\
where\\
k = lag\\
\gamma_k = covariance\ lag\ k\\
\mu = mean\ of\ the\ series\\
\sigma^2 = variance\ of\ the\ series = \frac{1}{n-1}\Sigma_{t = 1}^N (x_{t} - \mu) \cdot (x_{t} - \mu)$$

Notice that for any series, $\rho_0 = 1$. in other words, the autocorrleation of a series at lag zero equals one. 

We can also define a second order **partial autocorrelation**. The Partial autocoorelation at lag k is the correlation that results from removing the effect of any correlations due to the terms at smaller lags.

Let's plot the autocorrelation function (acf) and partial autocorrelation function (pacf) of the white noise series. Run the code in the cell below to compute and plot these functions. 

In [ ]:
import statsmodels.graphics.tsaplots as splt
splt.plot_acf(white, lags = 40)
splt.plot_pacf(white, lags = 40)

As expected the white noise series only has a significant autocorrelation and partial autocorrelation values at lag zero. There are no significant partial autocorrelation values. The shadded blue area on these plots shows the 95% confidence interval. 

****
**Note:** The Python statsmodels packages uses the engineering convention for displaying partial autocorrelation. The value at 0 lag, which always must be 1.0, is displayed. In many statistical packages, including R, this 0 lag value is not displayed. This difference in conventions can lead to a lot of confusion. 
****

**Your Turn:** In the cell below create and execute the code to create a monthly series `(freq = 'M')` of white noise plus two times a sinusodial component over a date range from January 2001 to January 2016. The white noise component should have mean 0 and standard deviation 0f 1.0. Create a time series plot of the result. Then plot the acf and pacf of the time series. **Hint:** You will need to use the `pi` constant from the `math` library. The sinusodial component will be `2 * sin(pi*x/6)`. Use `numpy.random.seed(6677)` to set the random number seed. 

Examine these results and answer the following questions:
1. Can you see the periodicity by negative and positive peaks at which lags of the ACF? ANS: negative at lag 6 and positive at lag 12. 
2. Can you see the periodicity by negative and positive peaks at which lags of the PACF? ANS: negative at lag 6 and positive at lag 12.
3. Does the periodic behavior of the ACF and PACF continue or die out? ANS: ACF has continued periodicity, PACF does not. 

### Random Walks

A **random walk** is the defined by the sum of a white noise series. In other words, the value of the random walk is the cumulative sum of the preceeding white noise series. 

$$y_t = y_{t-1} + w_t\\
Or\\
w_t = y_t - y_{t-1}$$ 

The quantities $w_t\ (y_t - y_{t-1})$ are known as the **innovations** of the random walk.

But note that the covariance of a random walk increases with time and is not bounded.

$$\gamma_k = Cov(x_t, x_{t+k}) = t \sigma^2 \rightarrow \infty\ as\ t \rightarrow \infty$$

Therefore, the random walk is **not stationary** . 

The code in the cell below computes a random walk series by taking the cumulative sum over Normally distributed inovations. Run this code and examine the results. 

In [ ]:
nr.seed(3344)
def ran_walk(start = '1-1990', end = '1-2015', freq = 'M', sd = 1.0, mean = 0):
    dates = pd.date_range(start = start, end = end, freq = freq)
    walk = pd.Series(nr.normal(loc = mean, scale = sd, size = len(dates)),
                    index = dates)
    return(walk.cumsum())
walk = ran_walk()   
plot_ts(walk, 'random walk')

The random walk wanders back and forth, but eventually heads in a particular direction. 

**Your Turn** What does the distribution of values of the random walk look like. What about the ACF and PACF of the random walk? In the cell below, create and execute the code to examine the probability distribution, ACF and PACF of the radom walk. 

Examine these results and answer the following questions:
1. How close to Normally distributed are the values of the random walk? ANS: The values are far from Normmaly distributed    
2. How are the properties of the ACF and PACF different from those of the white noise series? ANS: The ACF dies off very slowly, whereas the PACF has no significant non-zero lag values. 

### White Noise Series with Trend

What happens when we add a trend to the white noise series? Run the code in the cell below which adds a linear trend to a white noise series. 

In [ ]:
nr.seed(6677)
def trend(start = '1-1990', end = '1-2015', freq = 'M', slope = 0.02, sd = 1.0, mean = 0):
    dates = pd.date_range(start = start, end = end, freq = freq)
    trend = pd.Series([slope*x for x in range(len(dates))],
                    index = dates)
    trend = trend + nr.normal(loc = mean, scale = sd, size = len(dates))
    return(trend)
                              
trends = trend()   
plot_ts(trends, 'random walk')

As expected, the time series trends upward with a linear trend with iid Normal deviations. 

Run the code in the cell below to examine the distribution of values in this time series.

In [ ]:
dist_ts(trends, lab = '\n trend + white noise')

The distribution is mostly Normal, but with some skew. You can observe the skew as the 'S' shaped curve in the quantiles valuee.  

How does adding a trend change the ACF and PACF? Run the code in the cell below and examine the results. 

In [ ]:
splt.plot_acf(trends, lags = 40)
splt.plot_pacf(trends, lags = 40)

Note the that the ACF decays slowly, as was the case with the random walk. In additon, the PACF shows significant values for several lags. This is the result of the trend creating dependency from one value to the next. Any time series with a trend is **not stationary**.

### Time Series with a Seasonal Component

Many real-world tine series include a seasonal component. A seasonal component is a period variation in the values of the time series. The periods can be measured in years, months, days, days of the week, hours of the day, etc. Some examples of seasonal components of time series inclde:

- Opion expiration dates in capital markets.
- Annual holidays which can affect transportation, utility use, shopping habits, etc.
- Weekend vs. business days, which account for volumns of certian transaction behavior.
- Month of the year which can affect emploiyment patterns, weather, etc.

Let's investigate the properties of a time series with a seasonal component. The coded in the cell below creates and plots a time series with sinusoidal seasonal component with added White noise. Run this coded and examine the results.

In [ ]:
nr.seed(5544)
from math import pi
def seasonal_ts(start = '1-1990', end = '1-2015', freq = 'M', slope = 0.02, sd = 1.0, mean = 0):
    dates = pd.date_range(start = start, end = end, freq = freq)
    seasonal = pd.Series([slope*x for x in range(len(dates))],
                    index = dates)
    seasonal = seasonal + nr.normal(loc = mean, scale = sd, size = len(dates))
    seasonal = seasonal + [2.0*sin(pi*x/6) for x in range(len(dates))] + 5.0
    return(seasonal)

seasonal = seasonal_ts()
plot_ts(seasonal, 'seasonal data')

As expected the time series looks like a noisy sin wave with a trend.

**Your Turn** What does the distribution of values of the seasonal time series look like. What about the ACF and PACF of the sesonal time series? In the cell below, create and execute the code to examine the probability distribution, ACF and PACF of this time series. 

Answer the following questions:

- How close to Normally distributed are the values of the seasonal time series? ANS: The distribution is fairly close to Normal.
- How are the properties of the ACF and PACF different from those of time series you have already examined? ANS: The ACF decays slowly and shows strong periodic behavior and the PACF dies off realitively quickly and exhibits some periodic behavior. 

## Decomposition of Time Series

We have looked at the properties of several types of time series. 

- White noise series.
- Random walks.
- White noise series with trend.
- White noise series with seasonal component.

Next, we have to look into methods for decomposing time series data into its **trend, seasonal and residual** components.

### The STL Decomposition Models

A direct decomposition model is know as the **seasonal, trend and residual** or **STL** model. Not too surprisingly this model does the following:

- The trend is removed using a LOESS regression model. 
- The seasonal component is removed using a regression on periodic components.
- The remainder is know as the residual. 

The decomposition can be either **additive** or **multiplicative**. The additive model simply sums the componets and is writen:

$$TS(t) = S(t) + T(t) + R(t)$$

The multiplicative model multiplies the three components. This model is particuarly useful in the common case where the seasonal effect increases in proportional to the trend. We can write this  model as follows which is transforms to an addative model by a log transformation:  

$$TS(t) = S(t)\ *\ T(t)\ *\ R(t)\\
or\\
log(TS(t)) = S(t) + T(t) + R(t)$$

You can find details of this model in [Rob Hyndman's lecture notes](http://robjhyndman.com/uwafiles/5-Cross-validation.pdf). 

Let's try this out on a time series which has a seasonal, trend and white noise residual component. The code in the cell below models the trend and a monthly seasonal compoent. Run the code to compuute the model.

In [ ]:
import statsmodels.tsa.seasonal as sts
def decomp_ts(ts, freq = 'M', model = 'additive'):
    res = sts.seasonal_decompose(ts, model = model) #, freq = freq)
    #resplot = res.plot()
    res.plot()
    return(pd.DataFrame({'resid': res.resid, 
                         'trend': res.trend, 
                         'seasonal': res.seasonal},
                       index = ts.index) )

decomp = decomp_ts(seasonal)
print(decomp[:12])
decomp[-12:]
#res_mean = res_df.resid.mean()
#if abs(res_mean) > 0.1: res_df.resid = [x - res_mean for x in res_df.resid]

You can see the time series is now decomposed into the three components. Notice that the first and last 6 values of the trend and seasonal component are missing. We will need to take this into account when performing any analysis. 

**Your Turn:** In the cell below create and exectue the code to plot the ACF and PACF of the residual compontent for the STL decomposition.  **Hint**, the residual in the column three of the data frame retured by `ts.decomp`.

Does it appear that the STL process has removed the trend and seasonal components of the time series fully or partially? ANS: The ACF and PACF both decay rather rapidly, but  

## ARIMA Models for the Residual Series

Now that we have investigated the basic properties of time series and some decomposition methods, let's investigate models for dealing with the residuals.

The class of models we will investigate are known and **autoregressive integrated moving average** or **ARIMA** model. We will work our way through each component of this model in the next few subsections. 

The ARIMA model and its relatives are **linear** in their coefficients. ARIMA models are in effect linear regression models. However, as you will see, ARIMA models are constructed to account for the serial correlations in time series data.  

### Autoregressiive Model

The values of an **autoregrissive** or **AR** time series are determined by a linear combination of the past values. In other words, the AR model accounts for serial correlation in the values of the time series. We can write the value of an autoregressive series of **order p** or **AR(p)** series at time t as:

$$x_t = \alpha_1 x_{t-1} + \alpha_2 x_{t-2} \dots \alpha_p x_{t-p} + w_t\\
where\\
w_t = white\ noise\ at\ time\ t$$

An AR process has the following properties:

- $\rho_0 = 1$ always.
- $p_k = \alpha^k$
- Number of nonzero PACF values = p.

AR models are specifically for **stationary time series**. If the variance is not constant or the trend component has not been removed, AR models will not produce statisfactory results.

The function in the code in the cell below does the following:
1. Defines an ARMA proceess of the specified orders.
2. Tests the stationarity and invertability (stability) of the process.
3. Returns Pandas series with samples generated from the ARMA process. Note that the `generate_sample` funtion adds white noise with standard deviation of 1.0 unless otherwise specified. 

Run the code in the cell below which simulates and plots an AR(2), or $x_t = 0.75\ x_{t-1} = 0.25\ x_{t-2}$, model.

In [ ]:
nr.seed(4477)
import statsmodels.tsa.arima_process as arima
def ARMA_model(ar_coef, ma_coef, start = '1-2005', end = '1-2015'):
    dates = pd.date_range(start = start, end = end, freq = 'M')
    ts = arima.ArmaProcess(ar_coef, ma_coef)
    print('Is the time series stationary? ' + str(ts.isstationary))
    print('Is the time series invertable? ' + str(ts.isinvertible))
    return(pd.Series(ts.generate_sample(120), index = dates))
ts_series_ar2 = ARMA_model(ar_coef = [1, .75, .25], ma_coef = [1])

Notice that we have applied some tests to determine if our choice of $AR(2)$ coefficients are stable. 

Next, let's create a plot of this time series and have a look at the results. Execute the code in the cell below.

In [ ]:
def plot_ts(ts, title):
    ts.plot()
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Value')
plot_ts(ts_series_ar2, title = 'Plot of AR(2) process time series')

The values of this time series look fairly random. The series shows significant deviatiions from the zero, but has no trend. 

The question now is, what are the statistical properties of this $AR(2)$ process? Run the code in the cell below to plot the ACF and PACF of the AR(1) series. 

In [ ]:
splt.plot_acf(ts_series_ar2, lags = 40)
splt.plot_pacf(ts_series_ar2, lags = 40)

The AR(2) process produces a series with significant correlations in the lags, as shown in the  ACF plot. More importantly, the PACF has 2 significant non-zero lag values, consistent  with an AR(2) model. 

The AR time series model estimates the coeficients for the order of the model specified. The code in the cell below does the following:
1. Uses the `ARIMA` function from the `statsmodels` package to define model. The order of  the AR model is specified as `(p,0,0)`.
2. Fits the coefficient values using the `fit` method on the model object.
3. Prints the output of the `summary` method, showing useful statistics to understand the model.  

Run this code and examine  the  properties of the  results. 

In [ ]:
def model_ARIMA(ts, order):
    from statsmodels.tsa.arima_model import ARIMA
    model = ARIMA(ts, order = order)
    model_fit = model.fit(disp=0, method='mle', trend='nc')
    print(model_fit.summary())
    return(model_fit)
ar2_model = model_ARIMA(ts_series_ar2, order = (2,0,0))

Note the following about the AR model:

- The estimated AR coefficients have values farily close to the values used to generate the data. Further, true values are within the standard errors and confidence intervals of the estimated coefficients. Notice negative sign of the coefficient values. 
- The p-values are small and standard error ranges do not include 0 so the coeffieint values are significant. 

### Moving Average Model

For a **moving average** or **MA** model the value of the time series at time `t` is determined by a linear combination of past white noise terms. In other words, the MA model accounts for series correlation in the noise terms. We can write the MA(q) model as the linear combination of the last `q` white noise terms $w_i$:

$$x_t = w_t + \beta_1 w_{t-1} + \beta_2 w_{t-2} + \cdots + \beta_q w_{t-q}$$

An MA process has the following properties:

- $\rho_0 = 1$ always.
- Number of nonzero $\rho_k; k \ne 0$ values = q.

MA models are specifically for **stationary time series**. If the variance is not constant or the trend component has not been removed, MA models will not produce statisfactory results.

The code in the cell below computes an `MA(1)` model with a coeficient $\beta_1 = 0.75$, and plots the results. Run this code and examine the plot.

In [ ]:
ts_series_ma1 = ARMA_model(ar_coef = [1], ma_coef = [1, .75])
plot_ts(ts_series_ma1, title = 'Plot of MA(1) process time series')

The time series of the MA(1) process looks fairly random, with no trend. 

Next, execute the code in the cell below to plot the ACF and PACF of the MA(1) process. 

In [ ]:
splt.plot_acf(ts_series_ar2, lags = 40)
splt.plot_pacf(ts_series_ar2, lags = 40)

The ACF exhibits only one non-zero lag, which is expected for an MA(1) process. There are also some significant non-zero lags in the PACF, which is a result of random noise.  

Let's try to estimate the coefficients of the MA time series. The code in the cell below fits and MA(1) model to the time series. The model is specified as `(0,0,q)`. Execute this code and examine the result. 

In [ ]:
ma1_model = model_ARIMA(ts_series_ma1, order = (0,0,1))

Note the following about the AR model:

- The estimated MA coefficient has a value farily close to the value used to generate the data. Further, true value is within the standard error and confidence intervals of the estimated coefficient. 
- The p-values are small and standard error ranges do not include 0 so the coeffieint values are significant. 

### The Autoregressive Moving Average Model

We can combine the AR and MA models to create an **autoregressive moving averate** or **ARMA** model. This model accounts for serial correlation in both noise terms and values. We would expect we can write an ARMA model of order `(p,q)` as:

$$x_t = \alpha_1 x_{t-1} + \alpha_2 x_{t-2} \dots \alpha_p x_{t-p} +\\
w_t + \beta_1 w_{t-1} + \beta_2 w_{t-2} + \cdots + \beta_q w_{t-q}$$

The code in the cell below simulates and plots an `ARMA(1,1)` model. The model is specified by  `(p,0,q)`. Execute this code and examine the results. 

In [ ]:
ts_series_arma11 = ARMA_model(ar_coef = [1, .6], ma_coef = [1, .75])
plot_ts(ts_series_arma11, title = 'Plot of ARMA(1,1) process time series')

As expected, the ARMA(1,1) series has properties of both an AR(1) and MA(1) series. 

**Your Turn:** Try estimating the parameters of the time series, printing a summary of the model and plot the ACF and PACF of the residual. **Hint**, the ARMA(1,1) model is specified as `order = (1,0,1)`.  

Run your code, examine the results and answer these questions: 

- How do the estimated MA and AR coefficients compare to the orginal model? ANS: The estimated parameter values are close to the values used to generate the time series. 
- How do the standard errors of the coefficients compare to the magnitudes of the coefficients? ANS: The standard error is small compared to the coefficient values.
- Do the p-values and confidence intervals indicate the coefficients are significant. ANS: Yes, the p-values are small and the confidence intervals do not contain 0. 

### Autoregressive Integrated Moving Average Model

The **autoregreessive integrated moving average**, or **ARIMA** model adds an integrating term to the ARMA model. The integrating component performs differencing to model a random walk component. The integrating component models one of the **non-stationary** parts of a time series. The ARIMA model is defined by orders p, d, q. We have already looked at AR(p) and MA(q) models. The order of the differencing operator of the integrating term is defined by `d`. Since the integrating term is a differencing operator, there is no coeficient to estimate. 

In a pervious section we have already simulated a randomw walk series, and investigated its properties. The code in the cell below uses the Pandas `diff` method to perform first order differencing on the time series. This operation is an ARIMA(0,1,0) model. Execute this code and examine the plotted results. 

In [ ]:
walk_diff = walk.diff()
plot_ts(walk_diff, title = 'Plot of first order difference of random walk time series')

The random walk has been transformed. The time series does not wander any more.

**Your turn:** The question is what are the statistical properties of the remainder series are after the ARIMA(0,1,0) has been applied. Plot the distribution properties, the ACF and PACF of the remainder. **Hint:** You will need to remove the first value from the difference series since the difference operator cannot work on the first element of a time series. 

Examine your results and answer the following questions:
1. Are the values close to Normally distributed? ANS: Yes, the histogram and the q-q Normal plot both show close to Normal behavior. 
2. Are there any significant non-zero lag values in the ACF and PACF and what does this indicate. ANS: No, there are no significant non-zero lag values in either the ACF and PACF, indicating the remainder series is white noise. 

## Real Data Example

Let's apply the models we have been working with on some real-world data. We will work with a data set which shows the consumpton of chocolate, beer and electricity in Australia from 1958 to 1991. 

### Load and Examine the Data

As a first step, run the code in the cell below to load the data from the `.csv` file, add a time series index and examine the head and tail of the data frame. 

In [ ]:
CBE = pd.read_csv('cbe.csv')
CBE.index = pd.date_range(start = '1-1-1958', end = '12-31-1990', freq = 'M')
print(CBE.head())
print(CBE.tail())

As a next step we plot the three time series. Execute the code in the cell below to create these plots. 

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1)
CBE.choc.plot(ax = ax1)
CBE.beer.plot(ax = ax2)
CBE.elec.plot(ax = ax3)
ax1.set_ylabel('Choclate')
ax2.set_ylabel('Beer')
ax3.set_ylabel('Electric')
ax3.set_xlabel('Date')
ax1.set_title('Three Australian production time series')

Notice that for each of these time series the amplitude of the seasonal variation grows with time. This is a common situation with real world data. Seeing this situation indicates that we should use a **multiplicative decomposition model**.  

The multiplcative model can be easily transformed to an addative model by taking the logarithm of the values. The code in the cell below performs the log transformation and plots the result for the electric consumption time series. Execute this code and  examine the results.    

In [ ]:
import numpy as np
CBE['elec_log'] = np.log(CBE.elec)
plot_ts(CBE.elec_log, 'Log of Australian electric production')
CBE.columns

Notive the follwing properties about this time series.
- It has a a significant trend.
- The time series have a noticable seasonal component.
- The magnitude of the seasonal component increases with trend in the un-transformed time series. 
- The seasonal component of the log transformeed series has a nearly constant magnitude, but decreases a bit with time. 

These results indicate that an STL decomposition is required. Further, a multiplicative (log transformed) STL model is preferred. 

### STL Decomposition of Electric Time Series

Lets do some analysis of the electric time series. In this case, we will use a **multiplicative model** since the magnitude of the seasonal component generally increases with incresing trend. 

Execute the code in the cell below to compute the STL decomposition of the time series and examine the  results.

In [ ]:
elect_decomp = decomp_ts(CBE.elec_log)
print(elect_decomp.head(12))
print(elect_decomp.tail(12))

Note the following about these results:

- The periodic component looks reasonable, but may not be stationary as evidenced by the remainder.
- The removal of the trend component appears to be satisfactory.

We can apply the **Dicky Fuller test** to determine if the residual is stationary. The null hyposesis is that the time series is not stationary, but with no trend. 

The code in the cell below executes the DF test and prints some summary statistics. Notice that the first and last 6 elements of the must be filtered, since they have `nan` values. Execute this code and examine the results.  

In [ ]:
from statsmodels.tsa.stattools import adfuller
def DF_Test(ts):
    stationary = adfuller(ts)
    ## Print the results
    print('D-F statistic = ' + str(stationary[0]))
    print('p-value = ' + str(stationary[1]))
    print('number of lags used = ' + str(stationary[2]))
    print('Critical value at 5% confidence = ' + str(stationary[4]['5%']))
    print('Critical value at 10% confidence = ' + str(stationary[4]['10%']))
DF_Test(decomp.resid[6:-6])    

Given the DF statistic and p-value we can reject the null hypothesis that the residual is not stationary. 

As a next step, compute and plot the ACF and PACF for the remainder series.

In [ ]:
splt.plot_acf(decomp.resid[6:-6], lags = 40)
splt.plot_pacf(decomp.resid[6:-6], lags = 40)

The ACF and PACF exhibit both AR and MA behavior. However, there are signs of periodicity which the STL decomposition has not removed. 

### Apply ARIMA Model

Now that we have an STL decomposition of the electric use time series, we can compute an ARIMA model for the residual. As a starting point we will use an ARIMA(2,1,2) model. Run the code in the cell below and examine the results.

In [ ]:
arima_electric = model_ARIMA(decomp.resid[6:-6], order = (2,1,1))

The standard error of the second AR coefficient is similar in magnitude or larger than the coefficient themselve. Further, the confidence interval overlaps zero. This behavior indicates that the model is over fit or over parameterized.

Now, the question is, how do we find the 'best' ARIMA model for the residual? We need a criteria to compare models with different values of p, d and q. 

The **Baysian Information Criteria** or **BIC** is closely related to the Akaike Information Criteria. The BIC was proposed by Gideon Schwarz in 1978, and is sometimes referred to as the Schwarz Information Criteria. The BIC weights the number of parameters in the model by the log of the number of observations. We can write the BIC as:

$$BIC = ln(n)\ k- 2\ ln(\hat{L})\\
where\\
\hat{L} = the\ likelihood\ given\ the\ fitted\ model\ parmaters\ \hat\theta = p(x| \hat\theta)\\
x = observed\ data\\
k = number\ of\ model\ parameters\\
n = number\ of\ observations$$

The code in the iterates over a grid of p, d and q values. For each p, d, q combination the BIC is computed and compared to the best previous model. In a bit more detail the function in the cell below does the following:
1. Initialize a large value of BIC. 
2. Iterate over the grid of specified p, d and q.
3. Compute an ARIMA model of order (p,d,q). This process is wrapped in a `try` to prevent the function from crashing if the model is unstable and does not converge. 
4. The BIC of each model is compared to the best (lowest) BIC found so far. If a better model is found, that model, its parameters, and the BIC are saved as the best model.
5. Once the loop has completed the best BIC, the order of the best model and the best model are all returned. 

Execute the code and examine the results. 

In [ ]:
def model_ARIMA_2(ts, order):
    from statsmodels.tsa.arima_model import ARIMA
    from statsmodels.tsa.arima_model import ARIMAResults    
    model = ARIMA(ts, order = order)
    model_fit = model.fit(disp=0, method='mle', trend='nc')
    BIC = ARIMAResults.bic(model_fit)
    print('Testing model of order: ' + str(order) + ' with BIC = ' + str(BIC))
    return(BIC, order, model_fit)

def step_ARIMA(resid, p_max, d_max, q_max):
    from statsmodels.tsa.arima_model import ARIMAResults    
    from statsmodels.tsa.arima_model import ARIMA
    best_BIC = 9999999999999999.0
    for p in range(p_max + 1):
        for d in range(d_max + 1):
            for q in range(q_max + 1):
                if(p > 0 or q > 0):
                    try:
                        order = (p, d, q)
                        BIC, order, model = model_ARIMA_2(resid, order)
                        if(BIC < best_BIC):
                            best_model = model
                            best_BIC = BIC
                            best_order = order
                    except: 
                        pass        
    return(best_BIC, best_order, best_model) 

BIC, order, model = step_ARIMA(decomp.resid[6:-6], 3, 3, 3)    
print('***************************************')
print('Best model with BIC = ' + str(BIC) + ' and with order '+ str(order))

Examine the results above. You can see how the interation over the model order eventually converged to a model with the smallest BIC. 

**Your turn:** Having found an apparent best model order, but how well does this model fit the residuals? And, is the model actually invertable (stable)? In the cell below compute and print a summary of a model of the best order. 

Examine these result. Do all the coefficients appear to be significant, or is one coefficient of marginal significance, and if so which one? ANS: The second AR coefficient is of marginal significance. 

## Forecasting Time Series

Now that we have explored these data, the next step is to compute and evaluate a forecast model. In this case, we will hold back the last 12 months of data before we train the model. The final 12 months of data can then be used to evaluate the model. 

We will train the model in three steps:
1. Compute new features, the count of months from the start of the time series and the square of count of months. These features are used to model trend.
2. Normalize these numeric features.
3. Create new dummy (binary) variables for the months. These features are used to model the seasonal variation.
4. Coeffients for the trend and seasonal features are computed using a linear regression model.
5. An ARIMA model for the residuals is computed. 
6. A prediction of the electric produciton is made for a 12 month period using the trend, seasonal and residual models.

The code in the cell below performs the first three steps of the process. Execute this code and examine the results.

In [ ]:
## Create new features, the count of months from the start of the
## series and the square of the count of months.
CBE.loc[:, 'Month_Cnt'] = [float(i + 1) for i in range(len(CBE.elec_log))]
CBE.loc[:, 'Month_Cnt2'] = [x**2 for x in CBE.Month_Cnt]

## Normalize the time features
from scipy.stats import zscore
CBE.loc[:, ['Month_Cnt', 'Month_Cnt2']] = CBE.loc[:, ['Month_Cnt', 'Month_Cnt2']].apply(zscore)

## Create dummy variables for the months
years = int(len(CBE.elec_log)/12)
CBE.loc[:, 'Month'] = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'] * years
dummies = pd.get_dummies(CBE.loc[:, 'Month'])
CBE[list(dummies.columns)] = dummies
## Print the head of the data frame to look at the dummy variables.
CBE.head(12)

Examine the new features in the data frame. You can see the features for modeling the trend and the dummy variables for the seasonal variation.

The code in the cell below computes a linear model for coefficients of the trend and seasonal features. The steps are:

1. The features are extracted into a numpy array.
2. The lable is extracted into a numpy array.
3. The model is defined. Note that we do no use an intercept since we have the seasonal component, which is categorical feature. 
4. The model fit is computed.
5. Predictions of the trend and seasonal values are computed.
6. The residuals with respect to these predictions are computed. 

Execute this code.   

In [ ]:
import sklearn.linear_model as lm
X = CBE.loc[:'1989-12-31', ['Month_Cnt', 'Month_Cnt2', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].as_matrix()
Y = CBE.loc[:'1989-12-31', 'elec_log'].as_matrix()
lm_mod = lm.LinearRegression(fit_intercept = False)
mod_fit = lm_mod.fit(X, Y)

## and the predictions and the residual
CBE.loc[:'1989-12-31', 'scores'] = mod_fit.predict(X)
CBE.loc[:'1989-12-31', 'resids'] = CBE.loc[:'1989-12-31', 'scores'] - CBE.loc[:'1989-12-31', 'elec_log']

Let's have a look at how these predicted trend and seasonal components fit the actual electric production time series. The code in the cell below plots the actual time series in red and the values predicted by the trend and seasonal model in blue. Execute the code in the cell below and examine the results. 

In [ ]:
def plot_mod_fit(df, col):
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(8, 5)) # set plot area
    ax = fig.gca() # define axis  
    df.loc[:, col].plot(color = 'r', ax = ax)
    df.loc[:, 'scores'].plot(ax = ax) 
    ax.set_title('Actual ' + col + 'vs. the predicted values') # Give the plot a main title
    ax.set_xlabel('Date') # Set text for the x axis
    ax.set_ylabel(col)# Set text for y axis
plot_mod_fit(CBE, 'elec_log')   

You can see that the trend and seasonal components fit the actaul data fairly well. 

To have a point of comparison, exectute the code in the cell below to compute the **root mean square error (RMSE)** of the fit of the model for the last 12 months of the electric production time series. 

In [ ]:
def RMSE(ts, score):
    from math import sqrt
    return sqrt(np.std(ts - score))

## Make the forecast for the next year
X = CBE.loc['1989-12-31':, ['Month_Cnt', 'Month_Cnt2', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].as_matrix()
RMSE(CBE.loc['1989-12-31':, 'elec_log'].as_matrix(), mod_fit.predict(X))

Next, let's have a look at a time series plot and distribution plots of the residuals. Execute the code in the cell below and examine the results. 

In [ ]:
plot_ts(CBE.loc[:'1989-12-31', 'resids'], title = 'Residual time series of log electric production')
dist_ts(CBE.loc[:'1989-12-31', 'resids'], '\n residual of trend and seasonal model')

The time series plot of the residual shows some minor periodic behavior. The distribution plots show the values to be close to Normally distributed. 

One other question we can ask is if the residual series is stationary. The code in the cell below applys the Dicky Fuller test. Execute this code and examine the results. 

In [ ]:
DF_Test(CBE.loc[:'1989-12-31', 'resids'])

Notice that the results are marginal. We cannot reject the null hypotheis (the time series is not stationary) at the 95% confidence level, but can at the 90% confidence level. This should not be too surprising, give the periodic behavor seen in the time series plot. 

Now, we will compute an ARIMA model for the residual usig the `step_ARIMA` function we used earlier. We will train this model, without the last 12 months of data, which we will use to evaluate the model. Execute the code in the cell below to search for the best ARIMA mode order. 

In [ ]:
BIC, order, model_fit = step_ARIMA(CBE.loc[:'1989-12-31', 'resids'], 4, 3, 4)    
print('Best order = ' + str(order) + ' best BIC = ' + str(BIC))

Now that we have the best model order, we will compute an ARIMA model. We will then use the model to make predictions for the last 12 months of the electric production time series. Execute this code.

In [ ]:
start_index = len(CBE.loc[:'1989-12-31', 'resids'])
end_index = start_index + 12
model_prediction = model_fit.predict(start=start_index, end=end_index)
model_prediction

We are now ready to compute the prediction for the last 12 months of the log of the electric production time series. The prediction is the combination of the trend, seasonal and residual (ARIMA) models. The code in the cell below computes this prediction. Execute this code to compute the prediction. 

In [ ]:
## Make the forecast for the next year
X = CBE.loc['1989-12-31':, ['Month_Cnt', 'Month_Cnt2', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].as_matrix()
## and the predictions for the forecast
CBE.loc['1989-12-31':, 'scores'] =  mod_fit.predict(X) - model_prediction

Next execute the code in the cell below to plot the actual time series (in red) and the prediction (in blue). 

In [ ]:
plot_mod_fit(CBE, 'elec_log') 

The forecast looks reasonable. The red, actual time series, and the prediction, in blue, look very similar for the last 12 months. 

Finally, let's compute the RMSE and compare these results to the model with only trend and seasonal component. 

In [ ]:
RMSE(CBE.loc['1989-12-31':, 'elec_log'].as_matrix(), CBE.loc['1989-12-31':, 'scores'])

The RMSE has improved from 0.18 to 0.14. Adding the ARIMA component to the model has improved the prediction. 

## Models for Non-Stationary Variance.

The **Autoregressive conditional heteroskedasticity** or **ARCH** and **Generalized Autoregressive conditional heteroskedasticity** or **GARCH** model, and their many relatives, are specifically indended to deal with variance which changes with time. Robert Engle published the ARCH model in 1982 for which he was awarded the Nobel Prize in Economics in 2003. 

These models are beyond the scope of this lesson. Additional information can be found in the references given earlier. Software packages for these models are widely available, including in R and Python.  

####  Copyright 2017, Stephen F Elston. All right reserved. 